In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle

import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [3]:
def undistort(img):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def hls_transform(img, thresh):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    sat = hls[:,:,2]
    binary = np.zeros_like(sat)
    binary[(sat > thresh[0]) & (sat <= thresh[1])] = 1
    return binary

def abs_grad(img, sobel_kernel, thresh, orient):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary = np.zeros_like(scaled_sobel)
    binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary

def mag_grad(img, sobel_kernel, thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    binary = np.zeros_like(gradmag)
    binary[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return binary

def dir_grad(img, sobel_kernel, thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    graddir = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    binary = np.zeros_like(graddir)
    binary[(graddir >= thresh[0]) & (graddir <= thresh[1])] = 1
    return binary

def combined_gradient(img, ksize):
    gradx = abs_grad(img, sobel_kernel=ksize, thresh=(100, 255), orient='x')
    grady = abs_grad(img, sobel_kernel=ksize, thresh=(100, 255), orient='y')
    mag_binary = mag_grad(img, sobel_kernel=ksize, thresh=(100, 255))
    dir_binary = dir_grad(img, sobel_kernel=ksize, thresh=(np.pi/4, np.pi/2))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined

def pers_trans(img, src, dst):
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped

def histo(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    return histogram

In [ ]:
camera_calibration_values = open('cal_coeff.p', 'rb') 
camera_calibration_values = pickle.load(camera_calibration_values)
mtx = camera_calibration_values['mtx'] 
dist = camera_calibration_values['dist']

In [4]:
def lane_finder(img):
    #[mtx, dist] = pickle.load( open( "cal_coeff.p", "rb" ) )
    
    undist = img
    
    hls = hls_transform(undist, (100,255))
    grad = combined_gradient(undist, 3)
    
    mixed = np.zeros_like(grad)
    mixed[(grad == 1) | (hls == 1)] = 1
    
    img_size = (undist.shape[1], undist.shape[0])

    src = np.float32([[np.int(img_size[0]*0.435), np.int(img_size[1]*0.65)],
                      [np.int(img_size[0]*0.565), np.int(img_size[1]*0.65)],
                      [np.int(img_size[0]*0.9), img_size[1]],
                      [np.int(img_size[0]*0.1), img_size[1]]])
    offset = 150

    dst = np.float32([[offset, 0], 
                      [img_size[0]-offset, offset], 
                      [img_size[0]-offset, img_size[1]], 
                      [offset, img_size[1]]])
    
    warped = pers_trans(mixed,src,dst)
    histogram = histo(warped)
    
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped, warped, warped))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped.shape[0]/nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    if Left_Line.recent_xfitted:
        leftx_current = Left_Line.recent_xfitted[-1][-1]
    else:
        leftx_current = leftx_base
    
    if Right_Line.recent_xfitted:
        rightx_current = Right_Line.recent_xfitted[-1][-1]
    else:
        rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped.shape[0] - (window+1)*window_height
        win_y_high = warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    if len(left_lane_inds):
        Left_Line.allx = nonzerox[left_lane_inds]
        Left_Line.ally = nonzeroy[left_lane_inds]
        Left_Line.detected = True
    
    if len(right_lane_inds):
        Right_Line.allx = nonzerox[right_lane_inds]
        Right_Line.ally = nonzeroy[right_lane_inds]
        Right_Line.detected = True

    # Fit a second order polynomial to each
    Left_Line.current_fit = np.polyfit(Left_Line.ally, Left_Line.allx, 2)
    Right_Line.current_fit = np.polyfit(Right_Line.ally, Right_Line.allx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    left_fitx = Left_Line.current_fit[0]*ploty**2 + Left_Line.current_fit[1]*ploty + Left_Line.current_fit[2]
    right_fitx = Right_Line.current_fit[0]*ploty**2 + Right_Line.current_fit[1]*ploty + Right_Line.current_fit[2]
    
    if Left_Line.detected:
        if len(Left_Line.recent_xfitted)>10:
            Left_Line.recent_xfitted.pop(0)
        Left_Line.recent_xfitted.append(left_fitx)
    
    if Right_Line.detected:
        if len(Right_Line.recent_xfitted)>10:
            Right_Line.recent_xfitted.pop(0)
        Right_Line.recent_xfitted.append(right_fitx)
    
    Left_Line.bestx = sum(Left_Line.recent_xfitted)/len(Left_Line.recent_xfitted)
    Right_Line.bestx = sum(Right_Line.recent_xfitted)/len(Right_Line.recent_xfitted)
    
    # Define conversions in x and y from pixels space to meters
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    #position from center
    lane_center = 640
    right_distance = right_fitx[-1]-lane_center
    left_distance = lane_center - left_fitx[-1]
    
    if Left_Line.line_base_pos:
        if np.absolute(right_distance - left_distance) < 100:
            Right_Line.line_base_pos = right_distance
            Left_Line.line_base_pos = left_distance
    else:
        Right_Line.line_base_pos = right_distance
        Left_Line.line_base_pos = left_distance
    
    pos_from_center = np.absolute(Right_Line.line_base_pos - Left_Line.line_base_pos)*xm_per_pix

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(Left_Line.ally*ym_per_pix, Left_Line.allx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(Right_Line.ally*ym_per_pix, Right_Line.allx*xm_per_pix, 2)

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    if Left_Line.radius_of_curvature:
        if np.absolute(Left_Line.radius_of_curvature - left_curverad) < 400:
            Left_Line.radius_of_curvature = left_curverad
    else:
        Left_Line.radius_of_curvature = left_curverad
        
    if Right_Line.radius_of_curvature:
        if np.absolute(Right_Line.radius_of_curvature - right_curverad) < 400:
            Right_Line.radius_of_curvature = right_curverad
    else:
        Right_Line.radius_of_curvature = right_curverad
        
    pts_left = np.array([np.transpose(np.vstack([Left_Line.bestx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([Right_Line.bestx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    fitted_lane_image = np.zeros_like(warped)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(fitted_lane_image, np.int_([pts]), 1)

    unwarp = pers_trans(fitted_lane_image, dst, src)
    lanes=np.zeros_like(undist)
    lanes[(unwarp>0)]=(0,255,0)

    font = cv2.FONT_HERSHEY_SIMPLEX
    
    overlayed = cv2.addWeighted(undist, 1, lanes, 0.3, 0)

    left_text = 'Left lane curvature: %dm' %Left_Line.radius_of_curvature
    right_text = 'Right lane curvature: %dm' %Right_Line.radius_of_curvature
    pos_text = 'Position from lane center: %.3fm'%pos_from_center
    cv2.putText(overlayed, left_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.05)), font, 1, (0,255,0), 4, cv2.LINE_AA)
    cv2.putText(overlayed, right_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.1)), font, 1, (0,255,0), 4, cv2.LINE_AA)
    cv2.putText(overlayed, pos_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.15)), font, 1, (0,255,0), 4, cv2.LINE_AA)

    return overlayed
    

In [5]:
output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")

Left_Line = Line()
Right_Line = Line()

road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [04:21<00:00,  4.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 4min 22s


In [6]:
challenge_output = 'challenge_video_output.mp4'
clip1 = VideoFileClip("challenge_video.mp4")

Left_Line = Line()
Right_Line = Line()

road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challenge_video_output.mp4
[MoviePy] Writing video challenge_video_output.mp4


100%|████████████████████████████████████████| 485/485 [01:33<00:00,  5.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_output.mp4 

Wall time: 1min 34s


In [7]:
harder_challenge_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")

Left_Line = Line()
Right_Line = Line()

road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(harder_challenge_output, audio=False)

[MoviePy] >>>> Building video harder_challenge_video_output.mp4
[MoviePy] Writing video harder_challenge_video_output.mp4


100%|█████████████████████████████████████▉| 1199/1200 [04:15<00:00,  4.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_output.mp4 

Wall time: 4min 15s
